In [11]:
import simpy
import numpy as np

# Queuing System Simulation processes class
class QueuingSystemSimulation:
    def __init__(self, n, m, lambd, mu, v, env):
        # queuing system params
        self.n = n
        self.m = m
        self.lambd = lambd
        self.mu = mu
        self.v = v
        # empirical values lists
        self.L_queuing_system_list = []
        self.L_queue_list = []
        self.t_queuing_system_list = []
        self.t_queue_list = []
        # enviromental variables for simpy simulation
        self.env = env
        self.loader = simpy.Resource(env, n)

    def run(self):
        while True:
            yield self.env.timeout(np.random.exponential(1/self.lambd))
            self.env.process(self.make_request())

    def make_request(self):
        queque_len_before = len(self.loader.queue)
        n_busy = self.loader.count

        with self.loader.request() as request:
            self.L_queue_list.append(queque_len_before)
            self.L_queuing_system_list.append(queque_len_before + n_busy)
            if len(self.loader.queue) <= self.m:
                arrival_time = self.env.now

                waiting_process = self.env.process(self.waiting_in_queue())
                result = yield request | waiting_process    

                self.t_queue_list.append(self.env.now - arrival_time)
                if request in result:
                    yield self.env.process(self.request_processing())
                    self.t_queuing_system_list.append(self.env.now - arrival_time)
                else:
                    self.t_queuing_system_list.append(self.env.now - arrival_time)
            else:
                self.t_queue_list.append(0)
                self.t_queuing_system_list.append(0)

    def request_processing(self):
        yield self.env.timeout(np.random.exponential(1/self.mu))

    def waiting_in_queue(self):
        yield self.env.timeout(np.random.exponential(1/self.v))

In [13]:
def queuing_system_simulate(n, m, lambd, mu, v, test_time):
    env = simpy.Environment()
    qs = QueuingSystemSimulation(n, m, lambd, mu, v, env)
    env.process(qs.run())
    env.run(until=test_time)
    return qs.L_queuing_system_list, qs.L_queue_list, qs.t_queuing_system_list, qs.t_queue_list

L_queuing_system_list, L_queue_list, t_queuing_system_list, t_queue_list = queuing_system_simulate(3, 3, 2, 1, 1, 10000)
print(np.average(L_queuing_system_list), np.average(L_queue_list),
      np.average(t_queuing_system_list), np.average(t_queue_list))

1.982428352224917 0.20769192694154334 0.9858552241520535 0.10120679108804034
